Потрібно створити кілька моделей класифікації (логістична регресія, svm, k-mean, NBC, 
Decision Tree and Random Forest) для порівняння результатів.
Для того, щоб можна було порівнювати ваші результати, зробіть крос-валідацію на 4 фолди.
Підберіть параметри так, щоб отримати найкращі результати.

Про набір даних.
Цей набір даних отримано від Національного інституту діабету, захворювань органів травлення та нирок. Мета полягає в тому, щоб передбачити на основі діагностичних вимірювань, чи є у пацієнта діабет.

Кількість спостережень: 768
Кількість атрибутів: 8 плюс клас
Для кожного атрибута: (усі мають числові значення)
1. Кількість разів вагітності
2. Концентрація глюкози в плазмі через 2 години в оральному тесті на толерантність до глюкози
3. Діастолічний артеріальний тиск (мм рт. ст.)
4. Товщина шкірної складки трицепса (мм)
5. 2-годинний сироватковий інсулін (мкОд/мл)
6. Індекс маси тіла (вага в кг/(зріст у м)^2)
7. Діабет племінна функція
8. Вік (років)
9. Змінна класу (0 або 1)

In [73]:
import pandas as pd
diabetes_data = pd.read_csv('./diabetes.csv', sep=',', header=0)

# перевірити чи є пропущені дані
print('Кількість спостережень:', len(diabetes_data))
diabetes_data.dropna()
print('Кількість спостережень з цілісними даними:', len(diabetes_data))
diabetes_data


Кількість спостережень: 768
Кількість спостережень з цілісними даними: 768


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [7]:
# 1. Логістична регресія
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

X = diabetes_data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]

y = diabetes_data['Outcome'].values

# ініціалізуємо об'єкт класу, яким користуватимемося для розбивки даних
kf = KFold(
    n_splits=4,  # кількість окремих тестових наборів
    shuffle=True,  # чи перемішувати дані перед тим, як розділити на окремі фрагменти
    random_state=42  # фіксація випадкового стану
)
 

In [133]:
from sklearn.inspection import permutation_importance
def calculate_importance(mdl, X_test, y_test):
    test_res = [], [], []
    importances = permutation_importance(
        mdl, X_test, y_test,  # модель та дані, на яких оцінювати важливість
        n_repeats=10,  # скільки разів повторити перемішування та оцінку важливості
        random_state=42  # фіксація випадкового стану
        )
        
    for i, column in enumerate(X_test.columns):    
        # вивести на екран ім'я змінної, середнє значення та стандартне відхилення важливості змінної
        print(column, importances.importances_mean[i], importances.importances_std[i])

In [164]:
import numpy as np
from sklearn.metrics import mean_squared_error



# створюємо список моделей 
modelsList = { }

# перевіряємо метрики моделей для кожної моделі в списку
def evaluate_model(**mdl):
# ініціалізуємо порожні списки, в яких зберігатимемо значення помилки на кожному фолді
 train_res, test_res = [], []
 
 for modelName, model in mdl.items():
   # отримаємо індекси навчального та тестового набору
   for train_idx, test_idx in kf.split(X, y):
      # виберемо поточну навчальну та тестову групи
      X_train, y_train = X.loc[train_idx], y[train_idx]
      X_test, y_test = X.loc[test_idx], y[test_idx]
        
      # навчимо модель
      model.fit(X_train, y_train)

      # виберемо прогнози
      y_train_preds, y_test_preds = model.predict(X_train), model.predict(X_test)
    
      # порахуємо помилки та збережемо у відповідні списки
      train_res.append(np.round(mean_squared_error(y_train, y_train_preds), 2))
      test_res.append(np.round(mean_squared_error(y_test, y_test_preds), 2))
   calculate_importance(model, X_test, y_test)
  
   print(f'Помилки на навчальних наборах моделі {modelName}: ', train_res)
   print(f'Помилки на тестових наборах моделі {modelName}: ', test_res)

In [165]:
# ініціалізація Linear Regression моделі 
linRegMdl = LinearRegression()

# додаємо модель до списку
modelsList['Linear Regression'] = linRegMdl

In [166]:
# ініціалізація SVM моделі 
from sklearn.svm import SVC

svc = SVC()

# додаємо модель до списку
modelsList['Support Vector Machine'] = svc

In [1]:
# ініціалізація k-mean моделі 

In [2]:
evaluate_model(**modelsList)

NameError: name 'evaluate_model' is not defined

In [ ]:
# ініціалізація k-NBC моделі

In [ ]:
# ініціалізація Decision Tree моделі

In [ ]:
# ініціалізація Random Forest моделі